## Linear model

using a linear model to predict the energy consumption

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

from data_read import load_data_into_dataframe
from house_filtration import house_filtration_select

from datetime import datetime
import time

In [2]:
# Load the dataset
df = pd.read_csv('./data/ouput.csv')


In [3]:
df.head()

,LCLid,tstp,energy(kWh/hh)
0,MAC000168,2011-11-24 13:30:00,0.054
1,MAC000168,2011-11-24 14:00:00,0.178
2,MAC000168,2011-11-24 14:30:00,0.113
3,MAC000168,2011-11-24 15:00:00,0.068
4,MAC000168,2011-11-24 15:30:00,0.061


In [4]:
df["energy(kWh/hh)"] = pd.to_numeric(df["energy(kWh/hh)"], downcast="float", errors="coerce")
df['tstp'] = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in df['tstp']]


# id for the household we want to use to train and test
id_training = 'MAC000150' 
id_eval = 'MAC000152'


def find_household(identification):
    affluenthh_filt = df["LCLid"] == identification
    affluent_hh_data = df[affluenthh_filt]
    return affluent_hh_data

In [5]:
training = find_household(id_training)
eval = find_household(id_eval)

dftrain = training
dfeval = eval

y_train = dftrain.pop("energy(kWh/hh)")
y_eval = dfeval.pop("energy(kWh/hh)")

In [6]:
dftrain.head()

,LCLid,tstp
356842,MAC000150,2011-11-23 10:00:00
356843,MAC000150,2011-11-23 10:30:00
356844,MAC000150,2011-11-23 11:00:00
356845,MAC000150,2011-11-23 11:30:00
356846,MAC000150,2011-11-23 12:00:00


In [7]:
dftrain.shape

(39720, 2)

In [8]:
dftrain.describe(datetime_is_numeric=True)

,tstp
count,39720
mean,2013-01-10 05:01:03.690332416
min,2011-11-23 10:00:00
25%,2012-06-17 07:52:30
50%,2013-01-10 04:45:00
75%,2013-08-05 02:37:30
max,2014-02-28 00:00:00


In [9]:
y_train.head()

356842    0.301
356843    0.127
356844    0.201
356845    0.564
356846    0.159
Name: energy(kWh/hh), dtype: float32

In [10]:
CATEGORIAL_COLUMNS = ['LCLid']
NUMERIC_COLUMNS = ['tstp']

feature_columns = []
for feature_name in CATEGORIAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='LCLid', vocabulary_list=('MAC000150',), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='tstp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [11]:
#check https://www.tensorflow.org/tutorials/estimator/linear

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use


train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)


In [12]:
#Creating the model

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\denny\\AppData\\Local\\Temp\\tmpevzid2br', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


TypeError: Cannot convert the argument `type_value`: dtype('<M8[ns]') to a TensorFlow DType.